![Banner](images/banner.png)

# Calling PL/SQL

Documentation reference link: [Executing PL/SQL](https://python-oracledb.readthedocs.io/en/latest/user_guide/plsql_execution.html)

PL/SQL is a 'stored' procedural language that is stored and run inside the database itself. PL/SQL lets you capture business logic for reuse across all your applications.  You can call stored procedures and functions easily from python-oracledb.

In [ ]:
import oracledb

In [ ]:
un = "pythondemo"
pw = "welcome"
cs = "localhost/orclpdb1"

connection = oracledb.connect(user=un, password=pw, dsn=cs)

## PL/SQL Procedures

This shows the PL/SQL procedure `MYPROC` used in this demo:

In [ ]:
with connection.cursor() as cursor:
   
    cursor.execute("select dbms_metadata.get_ddl('PROCEDURE', 'MYPROC') from dual")
    ddl, = cursor.fetchone()
    print(ddl.read())

You can use `callproc()` to call the procedure.  Bind variables are passed by position:

In [ ]:
with connection.cursor() as cursor:

    myinvar  = 22
    myoutvar = cursor.var(int)    # allocate a 'variable' of integer type to hold the OUT bind parameter

    cursor.callproc('myproc', [myinvar, myoutvar])
    print(myoutvar.getvalue())

You can also call PL/SQL procedures via an 'anonymous' PL/SQL block.  This can be useful if you want to use named bind placeholders:

In [ ]:
with connection.cursor() as cursor:

    myinvar  = 33
    myoutvar = cursor.var(int)

    cursor.execute(' begin myproc(:v1, :v2); end;', {"v1": myinvar, "v2": myoutvar})
    print(myoutvar.getvalue())

## PL/SQL Functions

This shows the PL/SQL function `MYFUNC` used in this demo:

In [ ]:
with connection.cursor() as cursor:
    
    cursor.execute("select dbms_metadata.get_ddl('FUNCTION', 'MYFUNC') from dual")
    ddl, = cursor.fetchone()
    print(ddl.read())

You can use `callfunc()` to call the function. Bind variables are passed by position.  The second argument to `callfunc()` is the type of the PL/SQL function return value.  Here it is an integer:

In [ ]:
with connection.cursor() as cursor:

    data = "abc"
    id = 3
    res = cursor.callfunc('myfunc', int, (data, id))
    print(res)

Similar to calling PL/SQL procedures, you can also invoke PL/SQL procedures via an anonymous block, and optionally used named bind placeholders:

In [ ]:
with connection.cursor() as cursor:

    data  = "def"
    id = 4
    ret = cursor.var(int)

    cursor.execute(' begin :ret := myfunc(:data, :id); end;', {"ret": ret, "data": data, "id": id})
    print(ret.getvalue())

## REF CURSORS

REF CURSORS let result sets be returned to python-oracledb, commonly from PL/SQL.

Here is the PL/SQL procedure used in this example:

In [ ]:
with connection.cursor() as cursor:

    cursor.execute("""select text from all_source 
                      where name = 'MYREFCURSORPROC' and type = 'PROCEDURE' 
                      order by line""")
    rows = cursor.fetchall()
    for r, in rows:
        print(r, end="")

Use `callproc()` as shown before to call the PL/SQL procedure. The `ref_cursor` variable needs to be defined as a cursor so it can hold the returned REF CURSOR.  This second cursor is then simply iterated over exactly like a cursor for simple SELECT would be:

In [ ]:
with connection.cursor() as cursor:
    ref_cursor = connection.cursor()

    cursor.callproc("myrefcursorproc", (2, 6, ref_cursor))

    print("Rows between 2 and 6:")
    for row in ref_cursor:
        print(row)

## Implicit Cursors

Instead of binding a cursor to get a REF CURSOR, the `dbms_sql.return_result()` procedure can alternatively return a result set back which is fetched in python-oracledb using `getimplicitresults()`:

In [ ]:
with connection.cursor() as cursor:

    cursor.execute("""
            declare
                c1 sys_refcursor;
                c2 sys_refcursor;
            begin
                open c1 for
                select * from ParentTable;
                dbms_sql.return_result(c1);

                open c2 for
                select * from ChildTable;
                dbms_sql.return_result(c2);
            end;""")

    for resultSet in cursor.getimplicitresults():
        print("Result Set:")
        for row in resultSet:
           print(row)
